In [36]:
#https://pythonprogramming.net/static/downloads/machine-learning-data/titanic.xls
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
import numpy as np
from sklearn.cluster import MeanShift
from sklearn import preprocessing
import pandas as pd

'''
Pclass Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd)
survival Survival (0 = No; 1 = Yes)
name Name
sex Sex
age Age
sibsp Number of Siblings/Spouses Aboard
parch Number of Parents/Children Aboard
ticket Ticket Number
fare Passenger Fare (British pound)
cabin Cabin
embarked Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)
boat Lifeboat
body Body Identification Number
home.dest Home/Destination
'''

df = pd.read_excel(r'c:\Users\15764\Downloads\titanic.xls')
original_df = df.copy()
print(df.head())

   pclass  survived                                             name     sex  \
0       1         1                    Allen, Miss. Elisabeth Walton  female   
1       1         1                   Allison, Master. Hudson Trevor    male   
2       1         0                     Allison, Miss. Helen Loraine  female   
3       1         0             Allison, Mr. Hudson Joshua Creighton    male   
4       1         0  Allison, Mrs. Hudson J C (Bessie Waldo Daniels)  female   

       age  sibsp  parch  ticket      fare    cabin embarked boat   body  \
0  29.0000      0      0   24160  211.3375       B5        S    2    NaN   
1   0.9167      1      2  113781  151.5500  C22 C26        S   11    NaN   
2   2.0000      1      2  113781  151.5500  C22 C26        S  NaN    NaN   
3  30.0000      1      2  113781  151.5500  C22 C26        S  NaN  135.0   
4  25.0000      1      2  113781  151.5500  C22 C26        S  NaN    NaN   

                         home.dest  
0                     St 

In [37]:
 df.drop(['body','name','ticket','boat'],1, inplace=True)

In [38]:
df.describe()

,pclass,survived,age,sibsp,parch,fare
count,1309.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000
mean,2.294882,0.381971,29.881135,0.498854,0.385027,33.295479
std,0.837836,0.486055,14.413500,1.041658,0.865560,51.758668
min,1.000000,0.000000,0.166700,0.000000,0.000000,0.000000
25%,2.000000,0.000000,21.000000,0.000000,0.000000,7.895800
50%,3.000000,0.000000,28.000000,0.000000,0.000000,14.454200
75%,3.000000,1.000000,39.000000,1.000000,0.000000,31.275000
max,3.000000,1.000000,80.000000,8.000000,9.000000,512.329200


In [39]:
df.fillna(0,inplace=True)  ; df.head()

,pclass,survived,sex,age,sibsp,parch,fare,cabin,embarked,home.dest
0,1,1,female,29.0000,0,0,211.3375,B5,S,"St Louis, MO"
1,1,1,male,0.9167,1,2,151.5500,C22 C26,S,"Montreal, PQ / Chesterville, ON"
2,1,0,female,2.0000,1,2,151.5500,C22 C26,S,"Montreal, PQ / Chesterville, ON"
3,1,0,male,30.0000,1,2,151.5500,C22 C26,S,"Montreal, PQ / Chesterville, ON"
4,1,0,female,25.0000,1,2,151.5500,C22 C26,S,"Montreal, PQ / Chesterville, ON"


In [40]:
def handle_non_numerical_data(df):
    columns = df.columns.values    
    for column in columns:
        text_digit_vals = {}
        def convert_to_int(val):
            return text_digit_vals[val]
        
        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            column_contents = df[column].values.tolist()
            unique_elements = set(column_contents)
            x=0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    text_digit_vals[unique] = x
                    x+=1
            
            df[column] = list(map(convert_to_int,df[column]))
            
    return df
        
        

In [41]:
df = handle_non_numerical_data(df) ; df.head()

,pclass,survived,sex,age,sibsp,parch,fare,cabin,embarked,home.dest
0,1,1,1,29.0000,0,0,211.3375,127,2,132
1,1,1,0,0.9167,1,2,151.5500,160,2,171
2,1,0,1,2.0000,1,2,151.5500,160,2,171
3,1,0,0,30.0000,1,2,151.5500,160,2,171
4,1,0,1,25.0000,1,2,151.5500,160,2,171


In [42]:
X = np.array(df.drop(['survived'],1).astype(float)) 
X = preprocessing.scale(X)
Y = np.array(df['survived']) ; Y

array([1, 1, 0, ..., 0, 0, 0], dtype=int64)

In [43]:
clf = MeanShift()
clf.fit(X)

labels  = clf.labels_
cluster_centers = clf.cluster_centers_

In [44]:
original_df['cluster_group'] = pd.Series(labels)

In [45]:
n_clusters = len(set(labels)) ; n_clusters

4

In [46]:
survival_rates = {}

In [47]:
for i in range(n_clusters):
    temp_df = original_df[ (original_df['cluster_group']==float(i)) ]
    survival_cluster = temp_df[ temp_df['survived']==1 ]
    survival_rate = round(len(survival_cluster) / len(temp_df) * 100,2)
    survival_rates[i] = survival_rate

In [48]:
survival_rates

{0: 37.2, 1: 100.0, 2: 65.91, 3: 10.0}

In [55]:
original_df[ original_df['cluster_group'] == 1]

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,cluster_group
35,1,1,"Bowen, Miss. Grace Scott",female,45.0,0,0,PC 17608,262.3750,NaN,C,4,NaN,"Cooperstown, NY",1
49,1,1,"Cardeza, Mr. Thomas Drake Martinez",male,36.0,0,1,PC 17755,512.3292,B51 B53 B55,C,3,NaN,"Austria-Hungary / Germantown, Philadelphia, PA",1
50,1,1,"Cardeza, Mrs. James Warburton Martinez (Charlo...",female,58.0,0,1,PC 17755,512.3292,B51 B53 B55,C,3,NaN,"Germantown, Philadelphia, PA",1
183,1,1,"Lesurer, Mr. Gustave J",male,35.0,0,0,PC 17755,512.3292,B101,C,3,NaN,NaN,1
302,1,1,"Ward, Miss. Anna",female,35.0,0,0,PC 17755,512.3292,NaN,C,3,NaN,NaN,1


In [60]:
sr_for_first_class = round((original_df[ (original_df['pclass'] == 1) & ( original_df['survived'] == 1)].shape[0] /
                      original_df[ (original_df['pclass'] == 1)].shape[0] *100 ),2)
print(str(sr_for_first_class)+"%")

61.92%
